## Get MNIST dataset

In [3]:
import tensorflow as tf
import numpy as np
import input_data

## Initialize functions

In [4]:
def init_weights(shape):
    return tf.Variable(tf.random_normal(shape, stddev=0.01))


def model(X, w_h, w_o):
    h = tf.nn.sigmoid(tf.matmul(X, w_h)) # this is a basic mlp, think 2 stacked logistic regressions
    return tf.matmul(h, w_o) # note that we dont take the softmax at the end because our cost fn does that for us

## Initialize data

In [5]:
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)
trX, trY, teX, teY = mnist.train.images, mnist.train.labels, mnist.test.images, mnist.test.labels

Extracting MNIST_data/train-images-idx3-ubyte.gz


/home/ana/anaconda3/lib/python3.5/gzip.py:274: VisibleDeprecationWarning: converting an array with ndim > 0 to an index will result in an error in the future
  return self._buffer.read(size)
/home/ana/MyNumerai/model_example/input_data.py:42: VisibleDeprecationWarning: converting an array with ndim > 0 to an index will result in an error in the future
  data = data.reshape(num_images, rows, cols, 1)


Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


## Initialize Tensorflow

In [82]:
num_hidden = 1000
X = tf.placeholder("float", [None, 784])
Y = tf.placeholder("float", [None, 10])

w_h = init_weights([784, num_hidden]) # create symbolic variables
w_o = init_weights([num_hidden, 10])

In [83]:
py_x = model(X, w_h, w_o)

In [84]:
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(py_x, Y)) # compute costs
train_op = tf.train.GradientDescentOptimizer(0.05).minimize(cost) # construct an optimizer
predict_op = tf.argmax(py_x, 1)

## Run the model

In [85]:
sess = tf.Session()

In [86]:
# you need to initialize all variables
with sess.as_default():
    tf.initialize_all_variables().run()
    for i in range(5):
        for start, end in zip(range(0, len(trX), 128), range(128, len(trX)+1, 128)):
            sess.run(train_op, feed_dict={X: trX[start:end], Y: trY[start:end]})
        print(i, np.mean(np.argmax(teY, axis=1) ==
                         sess.run(predict_op, feed_dict={X: teX})))

0 0.7422
1 0.8333
2 0.8642
3 0.8792
4 0.8875


In [88]:
# you need to initialize all variables
with sess.as_default():
    tf.initialize_all_variables().run()
    print(sess.run(py_x, feed_dict={X: teX}))

[[ 0.04797831 -0.00612375  0.11340173 ..., -0.07337324  0.17766167
   0.20786572]
 [ 0.05612247 -0.00497966  0.11434293 ..., -0.07761528  0.1726777
   0.2137861 ]
 [ 0.04951853 -0.00759421  0.12068243 ..., -0.0818615   0.17571549
   0.21026406]
 ..., 
 [ 0.04969995 -0.00468035  0.10803118 ..., -0.07878245  0.17217587
   0.21815068]
 [ 0.05257744  0.00429524  0.11926924 ..., -0.08504998  0.18442351
   0.21322033]
 [ 0.0537012   0.00184983  0.12378798 ..., -0.07676299  0.17561927
   0.2214261 ]]


In [81]:
sess.close()